In [5]:
import ner as e

my_ner = e.NER(debug=True, init=False, save_to_db=True)

Database leaks exists


In [6]:
group1 = ['PMC10047700','PMC10054737']
group2 = ['PMC10087035','PMC10213353','PMC10226267',
                                 'PMC10263344','PMC10277951','PMC10310558','PMC10354928','PMC10359376',
                                 'PMC10364759','PMC10407498','PMC10416696','PMC10476123','PMC10626330',
                                 'PMC3432121','PMC4369843','PMC5209428','PMC5355508','PMC5542784',
                                 'PMC5579173','PMC5715046','PMC5808712','PMC5863981','PMC5900725',
                                 'PMC6331738','PMC6354192','PMC6394086','PMC6430759','PMC6433422']

group3 = [
                                 'PMC6449949','PMC6513788','PMC6795629','PMC6992684','PMC6997159',
                                 'PMC7103177','PMC7153184','PMC7199605','PMC7212129','PMC7227310',
                                 'PMC7320065','PMC7336323','PMC7397345','PMC7439541','PMC7457655',
                                 'PMC7489661','PMC7519527','PMC7528523','PMC7535676','PMC7539400',
                                 'PMC7580259','PMC7667929','PMC7765607','PMC7797855','PMC7801633',
                                 'PMC7814645','PMC7898229','PMC8126803','PMC8183725','PMC8208906',
                                 'PMC8229046','PMC8246753','PMC8346442','PMC8390156','PMC8476360',
                                 'PMC8489786','PMC8523496','PMC8683256','PMC8700187','PMC8724192',
                                 'PMC8758650','PMC8790330','PMC8793212','PMC8831416','PMC8840033',
                                 'PMC8863852','PMC8869881','PMC8887088','PMC9116683','PMC9161645',
                                 'PMC9199482','PMC9260864','PMC9262798','PMC9262801','PMC9276443',
                                 'PMC9300066','PMC9389786','PMC9423657','PMC9502513','PMC9523838',
                                 'PMC9541253','PMC9600250','PMC9635981','PMC9663802','PMC9891862',
                                 'PMC9909109']

In [8]:
entities = my_ner.process_texts(group2, True)


2024-01-19 11:42:52,982 - ner - DEBUG - title: Exposure to oestrogen and risk of anastomotic leakage after colorectal cancer surgery – A clue to the different leak rates in men and women
2024-01-19 11:42:52,982 - ner - DEBUG - abstract: Colorectal anastomotic leakage is consistently more common in men, regardless of tumour location. This fact is largely unexplained but might be a consequence of biological differences including hormonal exposure and not only related to anatomy.
2024-01-19 11:43:20,782 - ner - DEBUG - entities[0]: ['Colorectal', '-660', 'C0555952', 'spco', 0, 10]
2024-01-19 11:43:23,659 - ner - DEBUG - title: Indocyanine green fluorescence angiography decreases the risk of anastomotic leakage after rectal cancer surgery: a systematic review and meta-analysis
2024-01-19 11:43:23,661 - ner - DEBUG - abstract: Anastomotic leakage is a serious complication after rectal cancer resection. Intraoperative use of indocyanine green fluorescence angiography (ICGFA) can help prevent

error with file:  PMC10213353.txt


2024-01-19 13:15:09,957 - ner - DEBUG - title: Management of anastomotic leakage after robot-assisted minimally invasive esophagectomy with an intrathoracic anastomosis
2024-01-19 13:15:09,958 - ner - DEBUG - abstract: Anastomotic leakage is a feared complication after esophagectomy and associated with increased post-operative morbidity and mrotality. The aim of this study was to evaluate the management of leakage after robot-assisted minimally invasive esophagectomy (RAMIE) with intrathoracic anastomosis. From a single center prospectively maintained database, all patients with anastomotic leakages defined by the Esophageal Complications Consensus Group between 2016 and 2021 were included. Contained leakage was defined as presence of air or fluid at level of the anastomosis without the involvement of the mediastinum or thorax. Non-contained leakage was defined as mediastinitis and/or mediastinal/pleural fluid collections. The primary outcome was 90-day mortality and the secondary outc

In [3]:
import pandas as pd
from pyparsing import col

df = pd.DataFrame(entities, columns=['candidate', 'score', 'cui', 'semtype', 'start', 'end'])

In [4]:
df['start'] = df['start'].astype(int)
df['end'] = df['end'].astype(int)

In [15]:
df.head(20)

,candidate,score,cui,semtype,start,end
1,Anastomotic,-694,C0332854,spco,0,11
3,Anastomotic,-694,C0332854,spco,0,11
5,Anastomotic,-694,C0677554,topp,0,11
7,Anastomotic,-694,C0677554,topp,0,11
9,Anastomosis,-623,C0332853,acab,0,11
11,Anastomosis,-623,C0332853,acab,0,11
13,Anastomosis,-623,C0500470,bsoj,0,11
15,Anastomosis,-623,C0500470,bsoj,0,11
0,Anastomotic Leak,-1000,C0919691,patf,0,16
2,Leak,-861,C0332234,ftcn,12,16


In [5]:
end_counts = df.groupby('start')['end'].nunique()
print(len(end_counts[end_counts > 1]))
len(df.start.unique())

466


3544

In [9]:
# remove duplicates
df = df.drop_duplicates(subset=['start', 'end', 'cui', 'semtype'], keep='first')

# for all entries with the same start, end pair, sort by score and only keep up to 5
dff = df.groupby(['start', 'end']).apply(lambda x: x.sort_values(['score'], ascending=False)).reset_index(drop=True)
dff = dff.groupby(['start', 'end']).head(5)
dff

,candidate,score,cui,semtype,start,end
0,Anastomotic,-694,C0332854,spco,0,11
1,Anastomotic,-694,C0677554,topp,0,11
2,Anastomosis,-623,C0332853,acab,0,11
3,Anastomosis,-623,C0500470,bsoj,0,11
4,Anastomotic Leak,-1000,C0919691,patf,0,16
...,...,...,...,...,...,...
10823,Article,-770,C1706852,inpr,43983,43990
10824,Make,-737,C1881534,ftcn,44013,44017
10825,MANUFACTURER,-770,C0947322,inpr,44025,44037
10826,Not,-1000,C1518422,ftcn,44042,44045


In [13]:
df.groupby('start').agg({'end': 'nunique', 'candidate': 'first', 'cui': 'first', 'semtype': 'first'}).reset_index()

,start,end,candidate,cui,semtype
0,0,2,Anastomotic,C0332854,spco
1,12,1,Leak,C0332234,ftcn
2,27,2,Life,C0376558,idcn
3,44,2,Postoperative,C0032790,tmco
4,58,1,Complication,C0009566,patf
...,...,...,...,...,...
3539,43983,1,Article,C1706852,inpr
3540,44013,1,Make,C1881534,ftcn
3541,44025,1,MANUFACTURER,C0947322,inpr
3542,44042,1,Not,C1518422,ftcn


In [1]:
import db as db
database = db.Database()
database.start()



Database leaks exists


In [2]:
database.get_term_neighborhood('C0919691')

[('Anastomotic', 1, 'C0332854', 'spco', 0, 11, -694),
 ('investigate', 3, 'C1292732', 'ftcn', 3, 14, -1000),
 ('Study', 4, 'C0557651', 'mnob', 5, 10, -1000),
 ('Leakage', 2, 'C0015376', 'patf', 12, 19, -861),
 ('Characteristics', 3, 'C1521970', 'qlco', 19, 34, -1000),
 ('seriousness', 2, 'C0871902', 'inbe', 25, 32, -510),
 ('TO', 4, 'C0040363', 'geoa', 26, 28, -1000),
 ('lives', 1, 'C0595998', 'fndg', 27, 31, -562),
 ('Clinical', 2, 'C0205210', 'qlco', 33, 41, -581),
 ('Predictor', 3, 'C2698872', 'idcn', 39, 49, -737),
 ('Causes', 4, 'C1314792', 'ftcn', 41, 47, -1000),
 ('Problem', 2, 'C3854081', 'clna', 42, 49, -748),
 ('Complication, Postoperative', 1, 'C0032787', 'patf', 44, 70, -861),
 ('Diagnostic', 4, 'C0430533', 'diap', 49, 58, -928),
 ('Anastomosis', 3, 'C0500470', 'bsoj', 54, 65, -532),
 ('Colorectal resection', 2, 'C0810152', 'topp', 56, 77, -595),
 ('Complicated', 1, 'C0231242', 'ftcn', 58, 70, -741),
 ('Treatment', 4, 'C1533734', 'topp', 64, 73, -770),
 ('Leakage', 3, 'C001

In [9]:
database.connection.close()

In [11]:
database.get_text_as_dict_by_pmid("PMC9663802")

{'id': 103,
 'pmid': 'PMC9663802',
 'title': 'New understanding of gut microbiota and colorectal anastomosis leak: A collaborative review of the current concepts',
 'abstract': 'Anastomotic leak (AL) is a life-threatening postoperative complication following colorectal surgery, which has not decreased over time. Until now, no specific risk factors or surgical technique could be targeted to improve anastomotic healing. In the past decade, gut microbiota dysbiosis has been recognized to contribute to AL, but the exact effects are still vague. In this context, interpretation of the mechanisms underlying how the gut microbiota contributes to AL is significant for improving patients outcomes. This review concentrates on novel findings to explain how the gut microbiota of patients with AL are altered, how the AL-specific pathogen colonizes and is enriched on the anastomosis site, and how these pathogens conduct their tissue breakdown effects. We build up a framework between the gut microbiot

In [13]:
database.get_entities_as_dict(103)

[]

In [4]:
for pmid in database.get_pmids_without_entities():
    database.clean_pmid(pmid)